# Image Table Scanner

improving cell detection and ocr by changing kernel values

In the original modules, the smallest cells were not detected because they were smaller than the minimum threshold value for detection.
I solved this by decreasing the minimum threshold for detection.

Prior to ocr, the ocr_image module would clean the image of each cell. This led to certain numbers being cropped and others deleted entirely.
I fixed this by increasing the kernel heigth and width for detecting cell boundaries. Now numbers are not cleaned out.

In [ ]:
# probably not needed

from PIL import Image # same as above
from pdf2image import convert_from_path # to convert pdf to image

In [38]:
# Importing other necessary packages
import glob
import pandas as pd
import numpy as np
import cv2 # image transformation
import os
import re

from io import StringIO # to convert string to csv
import time # to measure time

# to add the path where to search for modules
import sys
sys.path.append('/home/hennes/Internship/table_scanner')

# Importing table_ocr modules 
from table_ocr import pdf_to_images
from table_ocr import extract_tables
from table_ocr import extract_cells
from table_ocr import ocr_image
from table_ocr import ocr_to_csv

## Creating Pipeline

In [86]:
folder = "/home/hennes/Downloads/2021 Form 20 Digitized Data/*"
pdflist = [pdf for pdf in glob.glob(folder) if pdf.endswith(".pdf")]

In [26]:
# Importing images

start = time.time()

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(pdf_to_images.pdf_to_images, pdflist)
    
end = time.time()
print('OCR time =', end - start)

OCR time = 32.20327162742615


In [102]:
imglist = [img for img in glob.glob(folder) if img.endswith('.png')]

In [28]:
# Preprocessing Table
start = time.time()

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(pdf_to_images.preprocess_img, imglist)

end = time.time()
print('OCR time =', end - start)

OCR time = 17.473884105682373


In [103]:
# Extracting Table Image from PDF Page image
start = time.time()

imglist = [[img] for img in imglist]

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(extract_tables.main, imglist)

end = time.time()
print('OCR time =', end - start)

OCR time = 1.2674717903137207


In [104]:
# list of individual tables in subfolders 

dirlist = [x[0] for x in
           [glob.glob(table) for table in 
           [directory+'/*' for directory in glob.glob(folder) if re.search(r'\d$', directory)]]]

In [105]:
# Extract individual cell images
start = time.time()

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(extract_cells.main, dirlist)

end = time.time()
print('OCR time =', end - start)

OCR time = 0.9457151889801025


In [122]:
# cells folder is first in list of two objects in individual image folders.
# That is why this code works (but only if executed after cells were extracted).

dirlist = [x[0]+'/*' for x in
           [glob.glob(table) for table in 
           [directory+'/*' for directory in glob.glob(folder) if re.search(r'\d$', directory)]]]

In [128]:
# perform OCR on each image

os.environ['OMP_THREAD_LIMIT'] = '1'
p_ocr_image = functools.partial(ocr_image.main, None)

celllists = [glob.glob(cellfolder) for cellfolder in dirlist]

for image_list in celllists:

    start = time.time()

    # perform OCR on each image
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        executor.map(p_ocr_image, image_list)

    end = time.time()
    print(f'Done with a table. Time needed =', end - start)

Done with a table. Time needed = 19.99060821533203
Done with a table. Time needed = 21.072968244552612
Done with a table. Time needed = 18.747819423675537
Done with a table. Time needed = 19.025944232940674
Done with a table. Time needed = 17.928096771240234
Done with a table. Time needed = 18.877747774124146
Done with a table. Time needed = 18.239675521850586
Done with a table. Time needed = 20.18060302734375
Done with a table. Time needed = 17.510886907577515
Done with a table. Time needed = 17.519128561019897
Done with a table. Time needed = 20.309592962265015
Done with a table. Time needed = 19.882492303848267
Done with a table. Time needed = 19.77414083480835
Done with a table. Time needed = 20.31639313697815
Done with a table. Time needed = 17.089356184005737
Done with a table. Time needed = 18.23711061477661
Done with a table. Time needed = 20.41466999053955
Done with a table. Time needed = 17.966264247894287
Done with a table. Time needed = 18.18431329727173
Done with a table. 

In [215]:
# get the names for the individual pages

pages = [filename for directory, filename in
         [os.path.split(x) for x in
          [folder for folder in glob.glob(folder) if re.search(r'\d$', folder)]]]

# create list of alphabetically ordered lists of ocred files

ocrlists = [sorted(y) for y in
            [glob.glob(f'/home/hennes/Downloads/2021 Form 20 Digitized Data/{x}/cells/ocr_data/*.txt') for x in pages]]
zippie = zip(pages, ocrlists)

In [216]:
# Put OCRed str into csv
gathered_data = []

for y, x in zippie:
    output = ocr_to_csv.main(x)
    csv = StringIO(output)
    print("Converting into csv from" + y)
    
    # Turning csv into dataframe
    # Skipping the first two rows because they have fewer columns than rest
    # Also useful for chaining of tables later
    df = pd.read_csv(csv,  header = None, skiprows=[0, 1])
    gathered_data.append(df)
    print("sucessfully appended data from" + y)

gathered_data = pd.concat(gathered_data)
gathered_data

Converting into csv fromAC1_Form20-005
sucessfully appended data fromAC1_Form20-005
Converting into csv fromAC1_Form20-000
sucessfully appended data fromAC1_Form20-000
Converting into csv fromAC2_Form20-013
sucessfully appended data fromAC2_Form20-013
Converting into csv fromAC2_Form20-016


ParserError: Error tokenizing data. C error: Expected 13 fields in line 4, saw 15


## Fixing the Cell Chaining

In [104]:
# list of individual tables in subfolders 

dirlist = [x[0] for x in
           [glob.glob(table) for table in 
           [directory+'/*' for directory in glob.glob(folder) if re.search(r'\d$', directory)]]]

In [105]:
# Extract individual cell images
start = time.time()

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(extract_cells.main, dirlist)

end = time.time()
print('OCR time =', end - start)

OCR time = 0.9457151889801025


In [ ]:
import cv2
import os

def extract_cell_images_from_table(image):
# First image is blurred to reduce noise
    BLUR_KERNEL_SIZE = (17, 17)
    STD_DEV_X_DIRECTION = 0
    STD_DEV_Y_DIRECTION = 0
    blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
# Then thresholded to facilitate transformations
    MAX_COLOR_VAL = 255
    BLOCK_SIZE = 15
    SUBTRACT_FROM_MEAN = -2
    
    img_bin = cv2.adaptiveThreshold(
        ~blurred,
        MAX_COLOR_VAL,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        BLOCK_SIZE,
        SUBTRACT_FROM_MEAN,
    )
# Finding Vertical and Horizontal Lines
    vertical = horizontal = img_bin.copy()
    SCALE = 5
    image_width, image_height = horizontal.shape
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
    horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
    vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
    
    horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
    vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))
    
    mask = horizontally_dilated + vertically_dilated
    
# Finding Contours of the lines
    contours, heirarchy = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
    )
    
    perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
    epsilons = [0.05 * p for p in perimeter_lengths]
    approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]
    
    # Filter out contours that aren't rectangular. Those that aren't rectangular
    # are probably noise.
    approx_rects = [p for p in approx_polys if len(p) == 4]
    bounding_rects = [cv2.boundingRect(a) for a in approx_polys]
    
    # Filter out rectangles that are too narrow or too short.
    MIN_RECT_WIDTH = 30
    MIN_RECT_HEIGHT = 10
    bounding_rects = [
        r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
    ]
    
    # The largest bounding rectangle is assumed to be the entire table.
    # Remove it from the list. We don't want to accidentally try to OCR
    # the entire table.
    largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
    bounding_rects = [b for b in bounding_rects if b is not largest_rect]
    
    cells = [c for c in bounding_rects]
    def cell_in_same_row(c1, c2):
        c1_center = c1[1] + c1[3] - c1[3] / 2
        c2_bottom = c2[1] + c2[3]
        c2_top = c2[1]
        return c2_top < c1_center < c2_bottom
    
    orig_cells = [c for c in cells]
    rows = []
    while cells:
        first = cells[0]
        rest = cells[1:]
        cells_in_same_row = sorted(
            [
                c for c in rest
                if cell_in_same_row(c, first)
            ],
            key=lambda c: c[0]
        )
    
        row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
        rows.append(row_cells)
        cells = [
            c for c in rest
            if not cell_in_same_row(c, first)
        ]
    
    # Sort rows by average height of their center.
    def avg_height_of_center(row):
        centers = [y + h - h / 2 for x, y, w, h in row]
        return sum(centers) / len(centers)
    
    rows.sort(key=avg_height_of_center)
    cell_images_rows = []
    for row in rows:
        cell_images_row = []
        for x, y, w, h in row:
            cell_images_row.append(image[y:y+h, x:x+w])
        cell_images_rows.append(cell_images_row)
    return cell_images_rows

def main(f):
    results = []
    directory, filename = os.path.split(f)
    table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    rows = extract_cell_images_from_table(table)
    cell_img_dir = os.path.join(directory, "cells")
    os.makedirs(cell_img_dir, exist_ok=True)
    paths = []
    for i, row in enumerate(rows):
        for j, cell in enumerate(row):
            cell_filename = "{:03d}-{:03d}.png".format(i, j)
            path = os.path.join(cell_img_dir, cell_filename)
            cv2.imwrite(path, cell)
            paths.append(path)
    return paths